In [52]:
# from google.colab import drive
# drive.mount('/content/drive/')

# 이론 (아래에 빈칸에 답을 서술하시오)

1. 이름 = {철수, 영희, 가영, 나영, 단군}, 성별 = {남, 여}, 나이 = {19, 20, 29, 39, 5000}
2. (철수, 남), (영희, 여), (가영, 여), (나영, 여), (단군, 남), (영희, 남), Dom(R) = {철수, 영희, 가영, 나영, 단군}, Im(R) = {남, 여}
3. 함수가 아니다. Dom(R) = 이름 은 만족시키지만, (영희, 여), (영희, 남) 모두 이 관계에 포함되면서 남 != 여 이기 때문에 함수가 아니다.

# 실습: approximate function

In [53]:
import IPython.display as ipd
import librosa
import librosa.display
import numpy as np

# filename = "/content/drive/MyDrive/ColabNotebooks/sound.wav" # google colab 용
filename = "sound.wav" 
data,sample_rate = librosa.load(filename)
ipd.Audio(filename)

In [54]:
print('data :', data[:5])
print('data size : ', len(data))

data : [0.         0.02578735 0.05151367 0.0770874  0.10244751]
data size :  11025


In [115]:
cf = [0, 0, 0, 0 ,0]

def func(cf) :
    time = np.linspace(0, 0.5, len(data))
    sound = np.sum(cf[0]*np.sin(np.array( [i*time for i in cf[1:]] )), axis=0)
    return sound

def mse(func):  # actually, this is MAE(Mean Absolute Error)
    error=np.abs(func()-data)
    return np.mean(error)*1500

def test() :
    return func(cf)

print('func error : ', mse(test))

func error :  211.1571269132653


In [116]:
# 여기서부터 코드를 작성하시오

init_cf = [max(data) / 4, 800, 1000, 1200, 1400]
# init_cf = [0., 0, 0, 0, 0]

# best_cf를 찾으시오

##################
# first attempt
best_cf = init_cf
err = 1000000000    # 초기 에러값
eps = 100   # 탐색 범위
eps2 = 1.1
trial = 160 # 시도횟수

for i in range(trial):
    c_t = np.array(best_cf.copy()) - eps
    c_t[0] += eps - (eps / 1000)

    for j in range(0, 5):
        eps_t = eps
        if j == 0:
            eps_t /= 10
        
        # eps를 줄여가며 best_cf 각 요소 +- eps 범위를 eps / 10 만큼 증가시키며 탐색
        for c in np.arange(0, eps_t * 2, eps_t / 10):
            cf = best_cf.copy()
            cf[j] = c_t[j] + c
            tmp = mse(test)
            if tmp < err:
                best_cf = cf
                err = tmp

    print("trial " + str(i+1) + " : " + str(eps))
    print(err, best_cf)
    eps /= eps2

cf = best_cf

err = mse(test)
print("final : " + str(err) + ", " + str(best_cf))
if err <= 5:
    print("pass the test")
else:
    print("fail the test")
##################

trial 1 : 100
187.3596405865512 [0.023023986816411934, 820.0, 1040.0, 1230.0, 1460.0]
trial 2 : 90.9090909090909
187.3596405865512 [0.023023986816411934, 820.0, 1040.0, 1230.0, 1460.0]
trial 3 : 82.64462809917354
187.3596405865512 [0.023023986816411934, 820.0, 1040.0, 1230.0, 1460.0]
trial 4 : 75.13148009015777
186.29893964513494 [0.023023986816411934, 820.0, 1032.4868519909842, 1230.0, 1460.0]
trial 5 : 68.30134553650706
184.5038910512058 [0.023023986816411934, 820.0, 1039.3169865446348, 1236.8301345536506, 1460.0]
trial 6 : 62.0921323059155
183.76979719655935 [0.023023986816411934, 820.0, 1033.1077733140432, 1230.620921323059, 1460.0]
trial 7 : 56.44739300537772
182.41883573143642 [0.023023986816411934, 820.0, 1038.752512614581, 1236.265660623597, 1460.0]
trial 8 : 51.31581182307065
181.01180898287797 [0.023023986816411934, 825.1315811823071, 1033.620931432274, 1236.265660623597, 1460.0]
trial 9 : 46.650738020973314
179.872109798552 [0.023023986816411934, 825.1315811823071, 1038.2860

In [126]:
# 여기서부터 코드를 작성하시오

init_cf = [max(data) / 4, 800, 1000, 1200, 1400]
# init_cf = [1, 1, 1, 1, 1]

best_cf = init_cf

time = np.linspace(0, 0.5, len(data))


def derivative_1():
    # ret = np.sum(np.sin(np.array([(c * time) for c in best_cf[1:]]))) * 1500 / len(data)
    # return ret if best_cf[0] > 0 else -ret
    return np.sum(np.sin(np.array([c * time for c in best_cf[1:]]))) * 1500 / len(data)


def derivative_2(c):
    return np.sum(best_cf[0] * time * np.cos(c, time)) * 1500 / len(data)


attempt = 100000
err = 1
learning_rate = 0.00001

for i in range(attempt):
    cf = [
        best_cf[0] - learning_rate * derivative_1(),
        best_cf[1] - learning_rate * derivative_2(best_cf[1]),
        best_cf[2] - learning_rate * derivative_2(best_cf[2]),
        best_cf[3] - learning_rate * derivative_2(best_cf[3]),
        best_cf[4] - learning_rate * derivative_2(best_cf[4])
    ]
    best_cf = cf
    tmp = mse(test)
    if tmp < err:
        break

    print(f"attempt : {i}, err : {tmp}, cf : {cf}")


attempt : 0, err : 287.26234980835346, cf : [0.12282144468674437, 800.0002067391248, 1000.0004650630322, 1199.9989662599821, 1399.9992496795003]
attempt : 1, err : 302.31000452922933, cf : [0.13686373226945556, 800.000543879506, 1000.0009292342146, 1199.9979350227243, 1399.9985019204214]
attempt : 2, err : 359.27300598716033, cf : [0.18601734060247244, 800.000919187027, 1000.001446468524, 1199.9967867751438, 1399.9976701432306]
attempt : 3, err : 428.57715908033464, cf : [0.2414901538640204, 800.0014288093099, 1000.0021494531286, 1199.9952274957407, 1399.996541869472]
attempt : 4, err : 464.638391016431, cf : [0.2693049076866763, 800.00208984147, 1000.0030620551157, 1199.993205610211, 1399.9950810608784]
attempt : 5, err : 433.66992614299085, cf : [0.24545116190692903, 800.0028261048423, 1000.0040797331981, 1199.9909543150463, 1399.9934576805858]
attempt : 6, err : 398.09714570814265, cf : [0.21749712261515236, 800.0034959170547, 1000.0050072256245, 1199.9889059660964, 1399.99198385760

KeyboardInterrupt: 

In [124]:
print(derivative_2(best_cf[1]))

-1.6909198120010243e-09
